In [ ]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import importlib
import time

from util import utils
from util import plotting
from stats import hbStats

In [ ]:
importlib.reload(plotting)
importlib.reload(hbStats)

### Load and prepare data

In [ ]:
dataFolder = "fitbit_analyzer\\dataDumpSubset"

In [ ]:
start = time.time()
hbData = utils.loadHBData(dataFolder)
end = time.time()
print("Data loaded in {:.2f}s".format(end - start))
data = pd.concat(hbData, ignore_index=True)
print("Sample")
print(data.head())

### Basic Overall Stats

In [ ]:
hbStats.basicStats(data)

### Basic Aggregated Stats

In [ ]:
hourStats = hbStats.groupByBasicStats(data['datetime'].dt.hour, data, 'hour')
plotting.plot(hourStats, ['count', 'max', 'min', 'std'], 'hour', 2, 2)
#dayStats = hbStats.groupByBasicStats(data['datetime'].dt.day, data, 'day')
#plotting.plot(dayStats, ['count', 'max', 'min', 'std'], 'day', 2, 2)
monthStats = hbStats.groupByBasicStats(data['datetime'].dt.month, data, 'month')
plotting.plotMonthlyStatsHb(monthStats)
dayOfWeekStats = hbStats.groupByBasicStats(data['datetime'].dt.dayofweek, data, 'weekday')
plotting.plotWeekdayStatsHb(dayOfWeekStats)

### Daily Plot

In [ ]:
nData = data.set_index(["datetime"])
print(nData)
nData.resample('1D').plot()
plt.show()

In [ ]:
plotting.plotDailyStatsHb(data)

### Min/Max Values

In [ ]:
hbStats.getMaxValues(data, 5)

In [ ]:
hbStats.getMinValues(data, 5)